In [1]:
import networkx as nx
import random
import heapq
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm import tqdm
import pandas as pd


# Đọc đồ thị từ file và gán xác suất ngẫu nhiên cho các cạnh
random.seed(6)
G = nx.DiGraph()

with open('email-Eu-core.txt/email-Eu-core.txt', 'r') as file:
    for line in file:
        if line.startswith('#'):
            continue
        source, target = map(int, line.split())
        probability = random.random()  # Xác suất ngẫu nhiên giữa 0 và 1
        weight = random.random()
        G.add_edge(source, target, probability=probability, weight=weight)

print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

# Xác định các tập S và T từ các cạnh của đồ thị
S = set()
T = set()
for u, v in G.edges():
    S.add(u)
    T.add(v)

# print("Set S:", S)
# print("Set T:", T)

# Hàm tính bậc mong đợi
def expected_degree_out(G, v):
    return sum(G[v][w]['weight'] *G[v][w]['probability'] for w in G.successors(v) if w in G)

def expected_degree_in(G, v):
    return sum(G[u][v]['weight'] *G[u][v]['probability'] for u in G.predecessors(v) if u in G)

# Hàm tính mật độ mong đợi
def expected_density_v3(G, S, T):
    if G.number_of_edges() == 0:
        return 0
    probability_sum = sum(G[u][v]['weight'] * G[u][v]['probability'] for u, v in G.edges() if u in S and v in T)
    return probability_sum / (len(S) * len(T))**0.5

# Hàm vẽ đồ thị
def plot_graph(G):
    pos = nx.spring_layout(G)  # Tạo bố cục cho đồ thị
    edge_labels = nx.get_edge_attributes(G, 'probability')  # Lấy thuộc tính xác suất của các cạnh

    # Vẽ đồ thị
    nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='red', node_size=500, font_size=8, font_weight='bold')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)  # Hiển thị nhãn của các cạnh

    plt.title('Uncertain Graph Visualization')
    plt.show()

    

# Hàm tính xác suất trung bình của các cạnh trong đồ thị
def average_edge_probability(G):
    total_probability = sum(nx.get_edge_attributes(G, 'probability').values())
    num_edges = G.number_of_edges()
    return total_probability / num_edges if num_edges > 0 else 0

# Hàm tính mật độ cạnh kỳ vọng của đồ thị
def calculate_edge_density(G):
    num_vertices = len(G.nodes())
    num_possible_edges = num_vertices * (num_vertices - 1) if num_vertices > 1 else 1
    sum_probabilities = sum(G[u][v]['probability'] for u, v in G.edges())
    return sum_probabilities / num_possible_edges


Number of nodes: 1005
Number of edges: 25571


In [2]:
def greedy_approximation(G, S, T, c):
    H = G.copy()
    best_graph = H.copy()
    S_temp = S.copy()
    T_temp = T.copy()
    highest_density = expected_density_v3(H, S_temp, T_temp)
    
    num_iterations = len(S_temp) + len(T_temp)  # Tổng số đỉnh ban đầu trong cả hai tập
    with tqdm(total=num_iterations, desc="Processing", unit="node") as pbar:
        while S_temp and T_temp:
            i_min = min(S_temp, key=lambda i: expected_degree_out(H, i))
            d_S = expected_degree_out(H, i_min)

            j_min = min(T_temp, key=lambda j: expected_degree_in(H, j))
            d_T = expected_degree_in(H, j_min)
            
            if (len(S_temp) > 0 and len(T_temp) > 0) and (c * d_S <= d_T / c):
                S_temp.remove(i_min)
                H.remove_node(i_min)
            else:
                T_temp.remove(j_min)
                H.remove_node(j_min)
            
            current_density = expected_density_v3(H, S_temp, T_temp)
            
            S_temp = {u for u in S_temp if u in H}
            T_temp = {v for v in T_temp if v in H}
            
            if current_density > highest_density:
                highest_density = current_density
                best_graph = H.copy()  # Lưu bản sao của H với mật độ cao nhất

            if not S_temp or not T_temp:
                break
            
            pbar.update(1)  # Cập nhật tiến trình mỗi khi một đỉnh được loại bỏ
    
    return best_graph

# Thực hiện thuật toán
c = len(S) / len(T)
uds_wiki_vote = greedy_approximation(G, S, T, c)
S_result = set(uds_wiki_vote.nodes())

# # Lấy các giá trị xác suất từ các cạnh
# probabilities = [G[u][v]['probability'] for u, v in G.edges()]
# weights = [G[u][v]['weight'] for u, v in G.edges()]
# # Tính giá trị trung bình
# mean_probability = np.mean(probabilities)
# mean_weight = np.mean(weights)


# # Tính độ lệch chuẩn
# std_deviation = np.std(probabilities)

    
# print("Giá trị cạnh trung bình của đồ thị protein579138:", mean_probability)
# print("Độ lệch chuẩn của của  đồ thị protein579138:", std_deviation)
# print("Trọng số cạnh trung bình của đồ thị là:",mean_weight)
# print("Mật độ cạnh kì vọng :",calculate_edge_density(G))
# print("Mật độ kì vọng:",expected_density_v3(G))

Processing:  54%|██████████████████████████████████▌                             | 1004/1859 [01:55<01:38,  8.67node/s]


In [3]:
# Lấy các giá trị xác suất từ các cạnh
probabilities = [uds_wiki_vote[u][v]['probability'] for u, v in uds_wiki_vote.edges()]
weights = [uds_wiki_vote[u][v]['weight'] for u, v in uds_wiki_vote.edges()]

# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)


# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)
print("Mật độ cạnh kì vọng của đồ thị con được tạo ra bởi thuật toán uds là:",calculate_edge_density(uds_wiki_vote))
print("Xác xuất cạnh trung bình của đồ thị con được tạo ra bởi thuật toán uds là:",average_edge_probability(uds_wiki_vote))
print("Độ lệch chuẩn của đồ thị con được tạo ra bởi thuật toán uds là:", std_deviation)
print("Trọng số cạnh trung bình của thuật toán obs là:",mean_weight)
print("số đỉnh của uds_wiki_vote:", len(uds_wiki_vote.nodes))
print("Số cạnh của uds_wiki_vote:", len(uds_wiki_vote.edges))

Mật độ cạnh kì vọng của đồ thị con được tạo ra bởi thuật toán uds là: 0.13707909777792984
Xác xuất cạnh trung bình của đồ thị con được tạo ra bởi thuật toán uds là: 0.504260449654959
Độ lệch chuẩn của đồ thị con được tạo ra bởi thuật toán uds là: 0.2890034563099639
Trọng số cạnh trung bình của thuật toán obs là: 0.5046069808358208
số đỉnh của uds_wiki_vote: 167
Số cạnh của uds_wiki_vote: 7536


# Xem các chỉ số cơ bản của đồ thị ban đầu 

In [ ]:

# Lấy các giá trị xác suất từ các cạnh
probabilities = [G[u][v]['probability'] for u, v in G.edges()]
weights = [G[u][v]['weight'] for u, v in G.edges()]
# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)


# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)

    
print("Giá trị cạnh trung bình của đồ thị protein579138:", mean_probability)
print("Độ lệch chuẩn của của  đồ thị protein579138:", std_deviation)
print("Trọng số cạnh trung bình của đồ thị là:",mean_weight)
print("Mật độ cạnh kì vọng :",calculate_edge_density(G))





In [8]:
def plot_graph(G):
    pos = nx.spring_layout(G)  # Tạo bố cục cho đồ thị
    edge_labels = nx.get_edge_attributes(G, 'probability')  # Lấy thuộc tính xác suất của các cạnh

    # Vẽ đồ thị
    nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='red', node_size=500, font_size=8, font_weight='bold')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)  # Hiển thị nhãn của các cạnh

    plt.title('Uncertain Graph Visualization')
    plt.show()
    
def plot_graph_v2(G):
    pos = nx.spring_layout(G)  # Tạo bố cục cho đồ thị

    # Lấy thuộc tính xác suất và trọng số của các cạnh
    edge_probabilities = nx.get_edge_attributes(G, 'probability')
    edge_weights = nx.get_edge_attributes(G, 'weight')

    # Kết hợp xác suất và trọng số vào nhãn cạnh
    edge_labels = {edge: f"{edge_weights[edge]:.2f}, {edge_probabilities[edge]:.2f}" for edge in G.edges()}

    # Vẽ đồ thị
    nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='red', node_size=200, font_size=8, font_weight='bold')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=7)  # Hiển thị nhãn của các cạnh

    plt.title('Uncertain Weighted Directed Graph Visualization')
    plt.show()
def average_edge_probability(G):
    """ Tính xác suất trung bình của các cạnh trong đồ thị G. """
    total_probability = sum(nx.get_edge_attributes(G, 'probability').values())
    num_edges = G.number_of_edges()
    return total_probability / num_edges if num_edges > 0 else 0
def calculate_edge_density(G):
    """ Tính mật độ cạnh kỳ vọng của đồ thị G. τ """
    num_vertices = len(G.nodes())
    num_possible_edges = num_vertices * (num_vertices - 1)  if num_vertices > 1 else 1
    sum_probabilities = sum(G[u][v]['probability'] for u, v in G.edges())
    return sum_probabilities / num_possible_edges

def average_edge_probability(G):
    """ Tính xác suất trung bình của các cạnh trong đồ thị G. """
    total_probability = sum(nx.get_edge_attributes(G, 'probability').values())
    num_edges = G.number_of_edges()
    return total_probability / num_edges if num_edges > 0 else 0

def calculate_edge_density_v2(G):
    """Tính mật độ cạnh kỳ vọng của đồ thị G. τ"""
    num_vertices = len(G.nodes())
    num_possible_edges = num_vertices * (num_vertices - 1) if num_vertices > 1 else 1
    sum_weighted_probabilities = sum(G[u][v]['weight'] * G[u][v]['probability'] for u, v in G.edges())
    return sum_weighted_probabilities / num_possible_edges
def expected_degree(G, v):
    return sum(G[u][v]['probability'] for u in G.predecessors(v) if u in G) + sum(G[v][w]['probability'] for w in G.successors(v) if w in G)

def expected_degree_v2(G, v):
    """Tính bậc kỳ vọng của một đỉnh v trong đồ thị G."""
    in_degree = sum(G[u][v]['weight'] * G[u][v]['probability'] for u in G.predecessors(v) if u in G)
    out_degree = sum(G[v][w]['weight'] * G[v][w]['probability'] for w in G.successors(v) if w in G)
    return in_degree + out_degree

# def expected_degree_out(G,v):
#     out_degree = sum(G[v][w]['weight'] * G[v][w]['probability'] for w in G.successors(v) if w in G)
# def expected_degree_in(G,v):
#     in_degree = sum(G[u][v]['weight'] * G[u][v]['probability'] for u in G.predecessors(v) if u in G)


def expected_degree_out(G,v):
    out_degree = sum(G[v][w]['probability'] for w in G.successors(v) if w in G)
def expected_degree_in(G,v):
    in_degree = sum(G[u][v]['probability'] for u in G.predecessors(v) if u in G)


def expected_density(G):
    if G.number_of_edges() == 0:
        return 0
    probability_sum = sum(G[u][v]['probability'] for u, v in G.edges())
    num_vertices = len(G.nodes())
    return probability_sum / num_vertices if num_vertices > 0 else 0

def expected_density_v3(G):
    if G.number_of_edges() == 0:
        return 0
    probability_sum = sum(G[u][v]['probability'] for u, v in G.edges())
    return probability_sum / (len(S) * len(T))**0.5 

def expected_density_v2(G):
    if G.number_of_edges() == 0:
        return 0
    # Tính tổng trọng số kỳ vọng của các cạnh
    weighted_probability_sum = sum(G[u][v]['weight'] * G[u][v]['probability'] for u, v in G.edges())
    num_vertices = len(G.nodes())
    return weighted_probability_sum / num_vertices if num_vertices > 0 else 0


def greedy_uds(G):
    """ Thuật toán GreedyUDS để tìm đồ thị con với mật độ kỳ vọng cao nhất """
    start_time = time.time()  # Ghi lại thời gian bắt đầu

    H = G.copy()
    best_graph = H.copy()
#     highest_density = expected_density_v2(H)
    highest_density = expected_density(H)
    
    # Tạo hàng đợi ưu tiên
    degree_cache = {v: expected_degree_v2(H, v) for v in H.nodes()}
    priority_queue = [(degree, v) for v, degree in degree_cache.items()]
    heapq.heapify(priority_queue)
    
    num_iterations = len(H.nodes()) - 2  # Số lần lặp
    iteration_start_time = time.time()  # Thời gian bắt đầu của vòng lặp
    
    with tqdm(total=num_iterations, desc="Processing", unit="node") as pbar:
        while len(H.nodes()) > 2:
            # Tìm đỉnh có bậc kỳ vọng thấp nhất
            while True:
                min_degree_vertex = heapq.heappop(priority_queue)[1]
                if min_degree_vertex in H:
                    break
            
            # Loại bỏ đỉnh có bậc kỳ vọng thấp nhất
            H.remove_node(min_degree_vertex)
            degree_cache.pop(min_degree_vertex, None)
            
            # Cập nhật bậc kỳ vọng của các đỉnh liền kề
            for u in list(G.predecessors(min_degree_vertex)):
                if u in H:
                    degree_cache[u] = expected_degree_v2(H, u)
                    heapq.heappush(priority_queue, (degree_cache[u], u))
            for w in list(G.successors(min_degree_vertex)):
                if w in H:
                    degree_cache[w] = expected_degree_v2(H, w)
                    heapq.heappush(priority_queue, (degree_cache[w], w))
            
            # Tính mật độ kỳ vọng hiện tại của H
#             current_density = expected_density_v2(H)
            current_density = expected_density(H)
            if current_density > highest_density:
                highest_density = current_density
                best_graph = H.copy()  # Lưu bản sao của H với mật độ cao nhất
            
            pbar.update(1)  # Cập nhật thanh tiến trình
    
    iteration_end_time = time.time()  # Thời gian kết thúc của vòng lặp
    end_time = time.time()  # Thời gian kết thúc tổng thể
    
    print(f"Thời gian tổng thể: {end_time - start_time:.4f} giây")
    print(f"Thời gian vòng lặp: {iteration_end_time - iteration_start_time:.4f} giây")
    
    return best_graph if best_graph is not None else H

def greedy_approximation(G, S, T, c):
    H = G.copy()
    best_graph = H.copy()
    S_temp = S
    T_temp = T
    highest_density = expected_density_v3(H)
    while S_temp and T_temp:
        i_min = min(S_temp, key=lambda i: expected_degree_out(H, i))
        d_S = expected_degree_out(H, i_min)
        
        j_min = min(T_temp, key=lambda j: expected_degree_in(H, j))
        d_T = expected_degree_in(H, j_min)
        
        if (len(S_temp) > 0 and len(T_temp) > 0) and (c * d_S <= d_T / c):
            S_temp.remove(i_min)
            H.remove_node(i_min)
        else:
            T_temp.remove(j_min)
            H.remove_node(j_min)
        
        current_density = expected_density_v3(H)
        
        if current_density > highest_density:
            highest_density = current_density
            best_graph = H.copy()  # Lưu bản sao của H với mật độ cao nhất

        if not S_tempS or not T_temp:
            break
    
    return best_graph



In [9]:
uds_wiki_vote = greedy_approximation(G, S, T, c)

TypeError: '<' not supported between instances of 'NoneType' and 'NoneType'

In [31]:
uds_wiki_vote = greedy_uds(G)

Processing: 100%|████████████████████████████████████████████████████████████████| 1003/1003 [01:31<00:00, 10.96node/s]

Thời gian tổng thể: 91.7502 giây
Thời gian vòng lặp: 91.5100 giây


In [9]:
# Lấy các giá trị xác suất từ các cạnh
probabilities = [uds_wiki_vote[u][v]['probability'] for u, v in uds_wiki_vote.edges()]
weights = [uds_wiki_vote[u][v]['weight'] for u, v in uds_wiki_vote.edges()]

# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)


# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)
print("Mật độ cạnh kì vọng của đồ thị con được tạo ra bởi thuật toán uds là:",calculate_edge_density(uds_wiki_vote))
print("Xác xuất cạnh trung bình của đồ thị con được tạo ra bởi thuật toán uds là:",average_edge_probability(uds_wiki_vote))
print("Độ lệch chuẩn của đồ thị con được tạo ra bởi thuật toán uds là:", std_deviation)
print("Trọng số cạnh trung bình của thuật toán obs là:",mean_weight)
print("số đỉnh của uds_wiki_vote:", len(uds_wiki_vote.nodes))
print("Số cạnh của uds_wiki_vote:", len(uds_wiki_vote.edges))

Mật độ cạnh kì vọng của đồ thị con được tạo ra bởi thuật toán uds là: 0.1071334327082849
Xác xuất cạnh trung bình của đồ thị con được tạo ra bởi thuật toán uds là: 0.5060789773006353
Độ lệch chuẩn của đồ thị con được tạo ra bởi thuật toán uds là: 0.28933197205858135
Trọng số cạnh trung bình của thuật toán obs là: 0.4980837340144134
số đỉnh của uds_wiki_vote: 215
Số cạnh của uds_wiki_vote: 9740


# Đối với đồ thị Wiki-Vote

In [33]:
random.seed(9)
# Initialize a directed graph
G = nx.DiGraph()

# Assuming your data is stored in a file called 'email-Eu-core.txt'
# and the data starts from a specific line after the header.
# Adjust the path to where you have stored your file.
with open('wiki-Vote.txt/Wiki-Vote.txt', 'r') as file:
    for line in file:
        # Skip header lines or process them differently if needed
        if line.startswith('#'):
            continue
        
        # Split the line into source and target nodes
        source, target = map(int, line.split())
        # Assign a random probability to each edge
        probability = random.random()  # Generate a random probability between 0 and 1
        # Assign a random weight to each edge (for example, between 1 and 10)
        weight = random.random()
        G.add_edge(source, target, probability=probability, weight=weight)

print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())




# Optional: Print some edges with their attributes to verify
# for u, v, data in G.edges(data=True):
#     print(f"Edge ({u}, {v}) - Probability: {data['probability']}, Weight: {data['weight']}")


# Lấy các giá trị xác suất từ các cạnh
probabilities = [G[u][v]['probability'] for u, v in G.edges()]
weights = [G[u][v]['weight'] for u, v in G.edges()]
# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)


# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)

    
print("Giá trị cạnh trung bình của đồ thị protein579138:", mean_probability)
print("Độ lệch chuẩn của của  đồ thị protein579138:", std_deviation)
print("Trọng số cạnh trung bình của đồ thị là:",mean_weight)
print("Mật độ cạnh kì vọng của đồ thị :",calculate_edge_density(G))





Number of nodes: 7115
Number of edges: 103689
Giá trị cạnh trung bình của đồ thị protein579138: 0.49914344764034696
Độ lệch chuẩn của của  đồ thị protein579138: 0.28897262501123644
Trọng số cạnh trung bình của đồ thị là: 0.4993515475524596
Mật độ cạnh kì vọng của đồ thị : 0.0010225140759015394


In [34]:
uds_wiki_vote = greedy_uds(G)

Processing: 100%|████████████████████████████████████████████████████████████████| 7113/7113 [45:34<00:00,  2.60node/s]

Thời gian tổng thể: 2735.2174 giây
Thời gian vòng lặp: 2734.3930 giây


In [35]:
# Lấy các giá trị xác suất từ các cạnh
probabilities = [uds_wiki_vote[u][v]['probability'] for u, v in uds_wiki_vote.edges()]
weights = [uds_wiki_vote[u][v]['weight'] for u, v in uds_wiki_vote.edges()]

# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)


# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)
print("Mật độ cạnh kì vọng của đồ thị con được tạo ra bởi thuật toán uds là:",calculate_edge_density(uds_wiki_vote))
print("Xác xuất cạnh trung bình của đồ thị con được tạo ra bởi thuật toán uds là:",average_edge_probability(uds_wiki_vote))
print("Độ lệch chuẩn của đồ thị con được tạo ra bởi thuật toán uds là:", std_deviation)
print("Trọng số cạnh trung bình của thuật toán obs là:",mean_weight)
print("số đỉnh của uds_wiki_vote:", len(uds_wiki_vote.nodes))
print("Số cạnh của uds_wiki_vote:", len(uds_wiki_vote.edges))

Mật độ cạnh kì vọng của đồ thị con được tạo ra bởi thuật toán uds là: 0.031152662403454906
Xác xuất cạnh trung bình của đồ thị con được tạo ra bởi thuật toán uds là: 0.5038790577648957
Độ lệch chuẩn của đồ thị con được tạo ra bởi thuật toán uds là: 0.2899912203713483
Trọng số cạnh trung bình của thuật toán obs là: 0.5027124756077233
số đỉnh của uds_wiki_vote: 789
Số cạnh của uds_wiki_vote: 38439


In [40]:
nx.write_gexf(uds_wiki_vote, "uds_wiki_vote.gexf")


# Đối với đồ thị p2p-Gnutella08

In [4]:
random.seed(1)
# Initialize a directed graph
G = nx.DiGraph()

# Assuming your data is stored in a file called 'email-Eu-core.txt'
# and the data starts from a specific line after the header.
# Adjust the path to where you have stored your file.
with open('p2p-Gnutella08.txt', 'r') as file:
    for line in file:
        # Skip header lines or process them differently if needed
        if line.startswith('#'):
            continue
        
        # Split the line into source and target nodes
        source, target = map(int, line.split())
        # Assign a random probability to each edge
        probability = random.random()  # Generate a random probability between 0 and 1
        # Assign a random weight to each edge (for example, between 1 and 10)
#         weight = random.randint(1, 10)
        weight = random.random()
        G.add_edge(source, target, probability=probability, weight=weight)

print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())




# Optional: Print some edges with their attributes to verify
# for u, v, data in G.edges(data=True):
#     print(f"Edge ({u}, {v}) - Probability: {data['probability']}, Weight: {data['weight']}")


# Lấy các giá trị xác suất từ các cạnh
probabilities = [G[u][v]['probability'] for u, v in G.edges()]
weights = [G[u][v]['weight'] for u, v in G.edges()]
# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)


# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)

    
print("Giá trị cạnh trung bình của đồ thị protein579138:", mean_probability)
print("Độ lệch chuẩn của của  đồ thị protein579138:", std_deviation)
print("Trọng số cạnh trung bình của đồ thị là:",mean_weight)
print("Mật độ cạnh kì vọng của đồ thị :",calculate_edge_density(G))





Number of nodes: 6301
Number of edges: 20777
Giá trị cạnh trung bình của đồ thị protein579138: 0.5004933345397413
Độ lệch chuẩn của của  đồ thị protein579138: 0.2896557323372045
Trọng số cạnh trung bình của đồ thị là: 0.4994369331921513
Mật độ cạnh kì vọng của đồ thị : 0.0002619576638561329


In [5]:
uds_wiki_vote = greedy_uds(G)

Processing: 100%|████████████████████████████████████████████████████████████████| 6299/6299 [07:05<00:00, 14.79node/s]

Thời gian tổng thể: 426.4292 giây
Thời gian vòng lặp: 425.9826 giây


In [6]:
# Lấy các giá trị xác suất từ các cạnh
probabilities = [uds_wiki_vote[u][v]['probability'] for u, v in uds_wiki_vote.edges()]
weights = [uds_wiki_vote[u][v]['weight'] for u, v in uds_wiki_vote.edges()]

# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)


# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)
print("Mật độ cạnh kì vọng của đồ thị con được tạo ra bởi thuật toán uds là:",calculate_edge_density(uds_wiki_vote))
print("Xác xuất cạnh trung bình của đồ thị con được tạo ra bởi thuật toán uds là:",average_edge_probability(uds_wiki_vote))
print("Độ lệch chuẩn của đồ thị con được tạo ra bởi thuật toán uds là:", std_deviation)
print("Trọng số cạnh trung bình của thuật toán obs là:",mean_weight)
print("số đỉnh của uds_wiki_vote:", len(uds_wiki_vote.nodes))
print("Số cạnh của uds_wiki_vote:", len(uds_wiki_vote.edges))

Mật độ cạnh kì vọng của đồ thị con được tạo ra bởi thuật toán uds là: 0.014396657948977414
Xác xuất cạnh trung bình của đồ thị con được tạo ra bởi thuật toán uds là: 0.5281214692808495
Độ lệch chuẩn của đồ thị con được tạo ra bởi thuật toán uds là: 0.2884720180566724
Trọng số cạnh trung bình của thuật toán obs là: 0.5263640825020626
số đỉnh của uds_wiki_vote: 302
Số cạnh của uds_wiki_vote: 2478


# Đối với đồ thị 579138.protein

In [24]:
random.seed(9)
# Đọc dữ liệu từ tệp TXT
file_path = '579138.protein.links.full.v12.0.txt/579138.protein.links.full.v12.0.txt'
data = pd.read_csv(file_path, sep=' ')

# Khởi tạo đồ thị có hướng
G = nx.DiGraph()

# Thêm các cạnh với trọng số và xác suất
for _, row in data.iterrows():
    protein1 = row['protein1']
    protein2 = row['protein2']
    combined_score = row['combined_score']
    
    # Giả sử xác suất được tính dựa trên combined_score
    probability = combined_score / 1000.0  # Điều chỉnh hệ số nếu cần thiết

     # Định hướng cạnh dựa trên một quy tắc ngẫu nhiên
    if random.choice([True, False]):
        G.add_edge(protein1, protein2, probability= random.random(), weight= random.randint(1, 10))
    else:
        G.add_edge(protein2, protein1, probability= random.random(), weight=random.randint(1, 10))

# # Hàm vẽ đồ thị
# def plot_graph(G):
#     pos = nx.spring_layout(G)  # Tạo bố cục cho đồ thị

#     # Lấy thuộc tính xác suất và trọng số của các cạnh
#     edge_probabilities = nx.get_edge_attributes(G, 'probability')
#     edge_weights = nx.get_edge_attributes(G, 'weight')

#     # Kết hợp xác suất và trọng số vào nhãn cạnh
#     edge_labels = {edge: f"{edge_weights[edge]:.2f}, {edge_probabilities[edge]:.2f}" for edge in G.edges()}

#     # Vẽ đồ thị
#     nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='red', node_size=500, font_size=8, font_weight='bold')
#     nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=7)  # Hiển thị nhãn của các cạnh

#     plt.title('Uncertain Weighted Directed Graph Visualization')
#     plt.show()

# # Gọi hàm để vẽ đồ thị
# plot_graph(G)

# # In các cạnh của đồ thị có hướng
# print("Edges in directed graph:")
# for edge in G.edges(data=True):
#     print(edge)
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())



Number of nodes: 1676
Number of edges: 160062


In [26]:

# Lấy các giá trị xác suất từ các cạnh
probabilities = [G[u][v]['probability'] for u, v in G.edges()]
weights = [G[u][v]['weight'] for u, v in G.edges()]
# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)
do_lech_chuan_trong_so = np.std(weights)


# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)

    
print("Giá trị cạnh trung bình của đồ thị protein579138:", mean_probability)
print("Độ lệch chuẩn của của  đồ thị protein579138:", std_deviation)
print("Độ lệch chuẩn trọng số của của  đồ thị protein579138:", do_lech_chuan_trong_so)

print("Trọng số cạnh trung bình của đồ thị là:",mean_weight)
print("Mật độ cạnh kì vọng của đồ thị :",calculate_edge_density(G))


Giá trị cạnh trung bình của đồ thị protein579138: 0.4999780444494622
Độ lệch chuẩn của của  đồ thị protein579138: 0.2886006115406647
Độ lệch chuẩn trọng số của của  đồ thị protein579138: 2.8761175362052445
Trọng số cạnh trung bình của đồ thị là: 5.501368219814822
Mật độ cạnh kì vọng của đồ thị : 0.028506923289520425


In [27]:
uds_wiki_vote = greedy_uds(G)

Processing:  72%|█████████████████████████████████████████████▉                  | 1202/1674 [12:34<04:56,  1.59node/s]


KeyboardInterrupt: 

In [ ]:
# Lấy các giá trị xác suất từ các cạnh
probabilities = [uds_wiki_vote[u][v]['probability'] for u, v in uds_wiki_vote.edges()]
weights = [uds_wiki_vote[u][v]['weight'] for u, v in uds_wiki_vote.edges()]

# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)


# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)
do_lech_chuan_trong_so = np.std(weights)
print("Mật độ cạnh kì vọng của đồ thị con được tạo ra bởi thuật toán uds là:",calculate_edge_density(uds_wiki_vote))
print("Xác xuất cạnh trung bình của đồ thị con được tạo ra bởi thuật toán uds là:",average_edge_probability(uds_wiki_vote))
print("Độ lệch chuẩn của đồ thị con được tạo ra bởi thuật toán uds là:", std_deviation)
print("Độ lệch chuẩn trọng số của đồ thị con được tạo ra bởi thuật toán uds là:", do_lech_chuan_trong_so)
print("Trọng số cạnh trung bình của thuật toán obs là:",mean_weight)
print("số đỉnh của uds_wiki_vote:", len(uds_wiki_vote.nodes))
print("Số cạnh của uds_wiki_vote:", len(uds_wiki_vote.edges))

## Thử với trọng số từ 0 tới 1 

In [4]:
random.seed(9)
# Đọc dữ liệu từ tệp TXT
file_path = '579138.protein.links.full.v12.0.txt/579138.protein.links.full.v12.0.txt'
data = pd.read_csv(file_path, sep=' ')

# Khởi tạo đồ thị có hướng
G = nx.DiGraph()

# Thêm các cạnh với trọng số và xác suất
for _, row in data.iterrows():
    protein1 = row['protein1']
    protein2 = row['protein2']
    combined_score = row['combined_score']
    
    # Giả sử xác suất được tính dựa trên combined_score
    probability = combined_score / 1000.0  # Điều chỉnh hệ số nếu cần thiết

     # Định hướng cạnh dựa trên một quy tắc ngẫu nhiên
    if random.choice([True, False]):
        G.add_edge(protein1, protein2, probability= probability, weight= random.random())
    else:
        G.add_edge(protein2, protein1, probability= probability, weight=random.random())

# # Hàm vẽ đồ thị
# def plot_graph(G):
#     pos = nx.spring_layout(G)  # Tạo bố cục cho đồ thị

#     # Lấy thuộc tính xác suất và trọng số của các cạnh
#     edge_probabilities = nx.get_edge_attributes(G, 'probability')
#     edge_weights = nx.get_edge_attributes(G, 'weight')

#     # Kết hợp xác suất và trọng số vào nhãn cạnh
#     edge_labels = {edge: f"{edge_weights[edge]:.2f}, {edge_probabilities[edge]:.2f}" for edge in G.edges()}

#     # Vẽ đồ thị
#     nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='red', node_size=500, font_size=8, font_weight='bold')
#     nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=7)  # Hiển thị nhãn của các cạnh

#     plt.title('Uncertain Weighted Directed Graph Visualization')
#     plt.show()

# # Gọi hàm để vẽ đồ thị
# plot_graph(G)

# # In các cạnh của đồ thị có hướng
# print("Edges in directed graph:")
# for edge in G.edges(data=True):
#     print(edge)
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())


# Lấy các giá trị xác suất từ các cạnh
probabilities = [G[u][v]['probability'] for u, v in G.edges()]
weights = [G[u][v]['weight'] for u, v in G.edges()]
# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)
do_lech_chuan_trong_so = np.std(weights)


# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)

    
print("Giá trị cạnh trung bình của đồ thị protein579138:", mean_probability)
print("Độ lệch chuẩn của của  đồ thị protein579138:", std_deviation)
print("Độ lệch chuẩn trọng số của của  đồ thị protein579138:", do_lech_chuan_trong_so)

print("Trọng số cạnh trung bình của đồ thị là:",mean_weight)

Number of nodes: 1676
Number of edges: 160416
Giá trị cạnh trung bình của đồ thị protein579138: 0.32051819020546574
Độ lệch chuẩn của của  đồ thị protein579138: 0.21817695582306657
Độ lệch chuẩn trọng số của của  đồ thị protein579138: 0.2884085389238608
Trọng số cạnh trung bình của đồ thị là: 0.4992733452267177


In [5]:
uds_wiki_vote = greedy_uds(G)

Processing: 100%|████████████████████████████████████████████████████████████████| 1674/1674 [09:02<00:00,  3.09node/s]

Thời gian tổng thể: 543.4428 giây
Thời gian vòng lặp: 542.1975 giây


In [6]:
# Lấy các giá trị xác suất từ các cạnh
probabilities = [uds_wiki_vote[u][v]['probability'] for u, v in uds_wiki_vote.edges()]
weights = [uds_wiki_vote[u][v]['weight'] for u, v in uds_wiki_vote.edges()]

# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)


# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)
do_lech_chuan_trong_so = np.std(weights)
print("Mật độ cạnh kì vọng của đồ thị con được tạo ra bởi thuật toán uds là:",calculate_edge_density(uds_wiki_vote))
print("Xác xuất cạnh trung bình của đồ thị con được tạo ra bởi thuật toán uds là:",average_edge_probability(uds_wiki_vote))
print("Độ lệch chuẩn của đồ thị con được tạo ra bởi thuật toán uds là:", std_deviation)
print("Độ lệch chuẩn trọng số của đồ thị con được tạo ra bởi thuật toán uds là:", do_lech_chuan_trong_so)
print("Trọng số cạnh trung bình của thuật toán obs là:",mean_weight)
print("số đỉnh của uds_wiki_vote:", len(uds_wiki_vote.nodes))
print("Số cạnh của uds_wiki_vote:", len(uds_wiki_vote.edges))

Mật độ cạnh kì vọng của đồ thị con được tạo ra bởi thuật toán uds là: 0.23410217451228932
Xác xuất cạnh trung bình của đồ thị con được tạo ra bởi thuật toán uds là: 0.746686911890492
Độ lệch chuẩn của đồ thị con được tạo ra bởi thuật toán uds là: 0.2725871803311424
Độ lệch chuẩn trọng số của đồ thị con được tạo ra bởi thuật toán uds là: 0.28770387216996784
Trọng số cạnh trung bình của thuật toán obs là: 0.5026702060259106
số đỉnh của uds_wiki_vote: 123
Số cạnh của uds_wiki_vote: 9352


In [16]:
nx.write_gexf(uds_wiki_vote, "uds_579138_protein.gexf")


# Đối với đồ thị protein 1736402.protein.links.v12.0


In [7]:
random.seed(9)
# Đọc dữ liệu từ tệp TXT
file_path = '1736402.protein.links.v12.0.txt'
data = pd.read_csv(file_path, sep=' ')

# Khởi tạo đồ thị có hướng
G = nx.DiGraph()

# Thêm các cạnh với trọng số và xác suất
for _, row in data.iterrows():
    protein1 = row['protein1']
    protein2 = row['protein2']
    combined_score = row['combined_score']
    
    # Giả sử xác suất được tính dựa trên combined_score
    probability = combined_score / 1000.0  # Điều chỉnh hệ số nếu cần thiết

     # Định hướng cạnh dựa trên một quy tắc ngẫu nhiên
        
    if random.choice([True, False]):
        G.add_edge(protein1, protein2, probability= probability, weight= random.random())
#     else:
#         G.add_edge(protein2, protein1, probability= probability, weight= random.random())

# # Hàm vẽ đồ thị
# def plot_graph(G):
#     pos = nx.spring_layout(G)  # Tạo bố cục cho đồ thị

#     # Lấy thuộc tính xác suất và trọng số của các cạnh
#     edge_probabilities = nx.get_edge_attributes(G, 'probability')
#     edge_weights = nx.get_edge_attributes(G, 'weight')

#     # Kết hợp xác suất và trọng số vào nhãn cạnh
#     edge_labels = {edge: f"{edge_weights[edge]:.2f}, {edge_probabilities[edge]:.2f}" for edge in G.edges()}

#     # Vẽ đồ thị
#     nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='red', node_size=500, font_size=8, font_weight='bold')
#     nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=7)  # Hiển thị nhãn của các cạnh

#     plt.title('Uncertain Weighted Directed Graph Visualization')
#     plt.show()

# # Gọi hàm để vẽ đồ thị
# plot_graph(G)

# # In các cạnh của đồ thị có hướng
# print("Edges in directed graph:")
# for edge in G.edges(data=True):
#     print(edge)
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())


# Lấy các giá trị xác suất từ các cạnh
probabilities = [G[u][v]['probability'] for u, v in G.edges()]
weights = [G[u][v]['weight'] for u, v in G.edges()]
# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)
do_lech_chuan_trong_so = np.std(weights)


# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)

    
print("Giá trị cạnh trung bình của đồ thị protein579138:", mean_probability)
print("Độ lệch chuẩn của của  đồ thị protein579138:", std_deviation)
print("Độ lệch chuẩn trọng số của của  đồ thị protein579138:", do_lech_chuan_trong_so)

print("Trọng số cạnh trung bình của đồ thị là:",mean_weight)

Number of nodes: 3082
Number of edges: 315247
Giá trị cạnh trung bình của đồ thị protein579138: 0.2812061907012597
Độ lệch chuẩn của của  đồ thị protein579138: 0.18194945319649977
Độ lệch chuẩn trọng số của của  đồ thị protein579138: 0.28855910003547514
Trọng số cạnh trung bình của đồ thị là: 0.49985353210339717


In [8]:
uds_wiki_vote = greedy_uds(G)

Processing: 100%|██████████████████████████████████████████████████████████████| 3080/3080 [1:24:28<00:00,  1.65s/node]

Thời gian tổng thể: 5071.6272 giây
Thời gian vòng lặp: 5068.1048 giây


In [9]:
# Lấy các giá trị xác suất từ các cạnh
probabilities = [uds_wiki_vote[u][v]['probability'] for u, v in uds_wiki_vote.edges()]
weights = [uds_wiki_vote[u][v]['weight'] for u, v in uds_wiki_vote.edges()]

# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)


# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)
do_lech_chuan_trong_so = np.std(weights)
print("Mật độ cạnh kì vọng của đồ thị con được tạo ra bởi thuật toán uds là:",calculate_edge_density(uds_wiki_vote))
print("Xác xuất cạnh trung bình của đồ thị con được tạo ra bởi thuật toán uds là:",average_edge_probability(uds_wiki_vote))
print("Độ lệch chuẩn của đồ thị con được tạo ra bởi thuật toán uds là:", std_deviation)
print("Độ lệch chuẩn trọng số của đồ thị con được tạo ra bởi thuật toán uds là:", do_lech_chuan_trong_so)
print("Trọng số cạnh trung bình của thuật toán obs là:",mean_weight)
print("số đỉnh của uds_wiki_vote:", len(uds_wiki_vote.nodes))
print("Số cạnh của uds_wiki_vote:", len(uds_wiki_vote.edges))

Mật độ cạnh kì vọng của đồ thị con được tạo ra bởi thuật toán uds là: 0.24670937166530363
Xác xuất cạnh trung bình của đồ thị con được tạo ra bởi thuật toán uds là: 0.6476070192782917
Độ lệch chuẩn của đồ thị con được tạo ra bởi thuật toán uds là: 0.2841739259525697
Độ lệch chuẩn trọng số của đồ thị con được tạo ra bởi thuật toán uds là: 0.29096404152563127
Trọng số cạnh trung bình của thuật toán obs là: 0.5024647974844263
số đỉnh của uds_wiki_vote: 179
Số cạnh của uds_wiki_vote: 12138


# 300267.protein.links.v12.0

In [10]:
random.seed(9)
# Đọc dữ liệu từ tệp TXT
file_path = '300267.protein.links.v12.0.txt'
data = pd.read_csv(file_path, sep=' ')

# Khởi tạo đồ thị có hướng
G = nx.DiGraph()

# Thêm các cạnh với trọng số và xác suất
for _, row in data.iterrows():
    protein1 = row['protein1']
    protein2 = row['protein2']
    combined_score = row['combined_score']
    
    # Giả sử xác suất được tính dựa trên combined_score
    probability = combined_score / 1000.0  # Điều chỉnh hệ số nếu cần thiết

     # Định hướng cạnh dựa trên một quy tắc ngẫu nhiên
        
    if random.choice([True, False]):
        G.add_edge(protein1, protein2, probability= probability, weight= random.random())
#     else:
#         G.add_edge(protein2, protein1, probability= probability, weight= random.random())

# # Hàm vẽ đồ thị
# def plot_graph(G):
#     pos = nx.spring_layout(G)  # Tạo bố cục cho đồ thị

#     # Lấy thuộc tính xác suất và trọng số của các cạnh
#     edge_probabilities = nx.get_edge_attributes(G, 'probability')
#     edge_weights = nx.get_edge_attributes(G, 'weight')

#     # Kết hợp xác suất và trọng số vào nhãn cạnh
#     edge_labels = {edge: f"{edge_weights[edge]:.2f}, {edge_probabilities[edge]:.2f}" for edge in G.edges()}

#     # Vẽ đồ thị
#     nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='red', node_size=500, font_size=8, font_weight='bold')
#     nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=7)  # Hiển thị nhãn của các cạnh

#     plt.title('Uncertain Weighted Directed Graph Visualization')
#     plt.show()

# # Gọi hàm để vẽ đồ thị
# plot_graph(G)

# # In các cạnh của đồ thị có hướng
# print("Edges in directed graph:")
# for edge in G.edges(data=True):
#     print(edge)
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())


# Lấy các giá trị xác suất từ các cạnh
probabilities = [G[u][v]['probability'] for u, v in G.edges()]
weights = [G[u][v]['weight'] for u, v in G.edges()]
# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)
do_lech_chuan_trong_so = np.std(weights)


# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)

    
print("Giá trị cạnh trung bình của đồ thị protein579138:", mean_probability)
print("Độ lệch chuẩn của của  đồ thị protein579138:", std_deviation)
print("Độ lệch chuẩn trọng số của của  đồ thị protein579138:", do_lech_chuan_trong_so)
print("Trọng số cạnh trung bình của đồ thị là:",mean_weight)
print("Mật độ cạnh kì vọng ban đầu của đồ thị:", calculate_edge_density(G))

Number of nodes: 4136
Number of edges: 299573
Giá trị cạnh trung bình của đồ thị protein579138: 0.2811085778758432
Độ lệch chuẩn của của  đồ thị protein579138: 0.17968720524154452
Độ lệch chuẩn trọng số của của  đồ thị protein579138: 0.2885328357603919
Trọng số cạnh trung bình của đồ thị là: 0.49987883587971643
Mật độ cạnh kì vọng ban đầu của đồ thị: 0.004924030367738544


In [11]:

uds_wiki_vote = greedy_uds(G)
# Lấy các giá trị xác suất từ các cạnh
probabilities = [uds_wiki_vote[u][v]['probability'] for u, v in uds_wiki_vote.edges()]
weights = [uds_wiki_vote[u][v]['weight'] for u, v in uds_wiki_vote.edges()]

# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)


# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)
do_lech_chuan_trong_so = np.std(weights)
print("Mật độ cạnh kì vọng của đồ thị con được tạo ra bởi thuật toán uds là:",calculate_edge_density(uds_wiki_vote))
print("Xác xuất cạnh trung bình của đồ thị con được tạo ra bởi thuật toán uds là:",average_edge_probability(uds_wiki_vote))
print("Độ lệch chuẩn của đồ thị con được tạo ra bởi thuật toán uds là:", std_deviation)
print("Độ lệch chuẩn trọng số của đồ thị con được tạo ra bởi thuật toán uds là:", do_lech_chuan_trong_so)
print("Trọng số cạnh trung bình của thuật toán obs là:",mean_weight)
print("số đỉnh của uds_wiki_vote:", len(uds_wiki_vote.nodes))
print("Số cạnh của uds_wiki_vote:", len(uds_wiki_vote.edges))

Processing: 100%|████████████████████████████████████████████████████████████████| 4134/4134 [35:45<00:00,  1.93node/s]


Thời gian tổng thể: 2148.3647 giây
Thời gian vòng lặp: 2145.5570 giây
Mật độ cạnh kì vọng của đồ thị con được tạo ra bởi thuật toán uds là: 0.21540614303728947
Xác xuất cạnh trung bình của đồ thị con được tạo ra bởi thuật toán uds là: 0.5481180380472124
Độ lệch chuẩn của đồ thị con được tạo ra bởi thuật toán uds là: 0.2859192792732919
Độ lệch chuẩn trọng số của đồ thị con được tạo ra bởi thuật toán uds là: 0.2886491702540732
Trọng số cạnh trung bình của thuật toán obs là: 0.49879835258580135
số đỉnh của uds_wiki_vote: 183
Số cạnh của uds_wiki_vote: 13089
